In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cx_Oracle

In [2]:
def get_data_from_db(query):
    conn = cx_Oracle.connect('hoseo', 'hoseo', 'localhost:1521/xe')
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [3]:
# 소비자물가지수 변동에 의한 화폐가치 계산
value_factors = {
    'value_year':[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020],
    'value_factor':[1.083, 1.069, 1.056, 1.049, 1.039, 1.019, 1.004, 1.000, 1.000]
}

vf_df = pd.DataFrame(value_factors)
# vf_df.info()

In [4]:
# 기본 데이터 탐색으로 뽑아놓은 월별 평당 가격 데이터
readD = pd.read_csv('sub-data-files/region_avg_price_per_pyung_month.csv')

readD.set_index('region', inplace=True)
readD = readD.T
readD.index = [ f'{s[2:6]}/{s[-2:]}' for s in readD.index ]
readD['전국'] = readD.mean(axis=1).astype('int64')
readD.reset_index(inplace=True)
# readD.head()

In [5]:
region_table = [
    ('강원도','gw'), ('경기도','gg'), ('경상남도','gsn'),
    ('경상북도','gsb'), ('광주광역시','gj'), ('대구광역시','dg'),
    ('대전광역시','dj'), ('부산광역시','bs'), ('서울특별시','so'),
    ('세종특별자치시','sj'), ('울산광역시','us'), ('인천광역시','ic'),
    ('전라남도','jrn'), ('전라북도','jrb'), ('제주특별자치도','jj'),
    ('충청남도','ccn'), ('충청북도','ccb'), ('전국','all')
]

In [6]:
# eco_col = [
#     '일자', '코스피', '코스피200', '코스피200건설', '코스닥', '다우존스', 's&p500',
#     '상해종합', '니케이', '원달러환율', '금현물', '원유현물', '수출액', '수입액',
#     '생활물가지수', '소비자물가지수(주택수도전기연료)', '소비자심리지수',
#     '가계대출금리', '주택담보대출금리', '강원도p', '경기도p', '경상남도p', '경상북도p',
#     '광주광역시p', '대구광역시p', '대전광역시p', '부산광역시p', '서울특별시p',
#     '세종특별자치시p', '울산광역시p', '인천광역시p', '전라남도p', '전라북도p',
#     '제주특별자치도p', '충청남도p', '충청북도p', '전국_re_cc', '서울특별시_re_cc',
#     '인천광역시_re_cc', '경기도_re_cc', '부산광역시_re_cc', '대구광역시_re_cc',
#     '광주광역시_re_cc', '대전광역시_re_cc', '울산광역시_re_cc', '강원도_re_cc',
#     '충청북도_re_cc', '충청남도_re_cc', '전라북도_re_cc', '전라남도_re_cc',
#     '경상북도_re_cc', '경상남도_re_cc', '강원_loan', '경기_loan', '경남_loan',
#     '경북_loan', '광주_loan', '대구_loan', '대전_loan', '부산_loan', '서울_loan',
#     '세종_loan', '울산_loan', '인천_loan', '전남_loan', '전북_loan', '제주_loan',
#     '충남_loan', '충북_loan', '전국_loan'
# ]

eco_col = [
    'date_ym', 'kospi', 'kospi200', 'kospi200construct', 'kosdaq', 'dowjones', 's&p500',
    'shanghai', 'nikkei', 'exchange_rate', 'gold', 'oil', 'export', 'import',
    'cpi_living', 'cpi', 'ccsi',
    'loan_interest_rate', 'housing_loan_rate', 'gw_pop', 'gg_pop', 'gsn_pop', 'gsb_pop',
    'gj_pop', 'dg_pop', 'dj_pop', 'bs_pop', 'so_pop',
    'sj_pop', 'us_pop', 'ic_pop', 'jrn_pop', 'jrb_pop',
    'jj_pop', 'ccn_pop', 'ccb_pop', 'all_re_ccsi', 'so_re_ccsi',
    'ic_re_ccsi', 'gg_re_ccsi', 'bs_re_ccsi', 'dg_re_ccsi',
    'gj_re_ccsi', 'dj_re_ccsi', 'us_re_ccsi', 'gw_re_ccsi',
    'ccb_re_ccsi', 'ccn_re_ccsi', 'jrb_re_ccsi', 'jrn_re_ccsi',
    'gsb_re_ccsi', 'gsn_re_ccsi', 'gw_loan', 'gg_loan', 'gsn_loan',
    'gsb_loan', 'gj_loan', 'dg_loan', 'dj_loan', 'bs_loan', 'so_loan',
    'sj_loan', 'us_loan', 'ic_loan', 'jrn_loan', 'jrb_loan', 'jj_loan',
    'ccn_loan', 'ccb_loan', 'all_loan'
]


In [7]:
ecoD = pd.read_csv('sub-data-files/economic_ex.csv')
ecoD.columns = eco_col
# ecoD.info()


In [8]:
def make_csv_from_db(region, save_path):
    
    query = f"""
        SELECT region, region_sub, apt_name,
            apt_size, apt_floor, contract_date,
            contract_price, completion_year,
            (to_number(to_char(contract_date, 'YYYY')) - completion_year) period_completed,
            round((apt_size / 3.3), 2) pyung,
            round(contract_price / (apt_size / 3.3)) price_per_pyung
        FROM apt_deal_price
        WHERE region = '{region}'
        """

    dataD = get_data_from_db(query)
    lower_col = dataD.columns.values.tolist()
    dataD.columns = [ l.lower() for l in lower_col ]

    # 평당 현재 가격 정보 추가
    dataD['contract_year'] = dataD['contract_date'].dt.year
    dataD = dataD.merge(vf_df, how='left', left_on='contract_year', right_on='value_year')
    dataD['price_per_pyung_present'] = round(dataD['price_per_pyung'] * dataD['value_factor'])

    # 추가 데이터 머지를 위한 키값 추가
    dataD['contract_date_key'] = dataD['contract_date'].dt.strftime('%Y/%m')

    # 지역별 평당 가격 추가
    dataD = dataD.merge(readD, how='left', left_on='contract_date_key', right_on='index')

    # 지역별 평당 가격 비율 추가
    for r in region_table:
        dataD[f'{r[1]}_ratio'] = round(dataD[r[0]] / dataD[region], 2)

    # 지역별 평당 가격 컬럼 제거
    dataD.drop(columns= [ re[0] for re in region_table ], inplace=True)
    
    # 경제 데이터 추가
    dataD = dataD.merge(ecoD, how='left', left_on='contract_date_key', right_on='date_ym')
    
    dataD.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f'complete!!! {save_path}')


In [9]:
# test function
make_csv_from_db('강원도', 'region-data-files/gw_region.csv')

complete!!! region-data-files/gw_region.csv


In [11]:
# batch jobs
# for r in region_table[:-2]:
#     make_csv_from_db(r[0], f'region-data-files/{r[1]}_region.csv')

In [10]:
tt = pd.read_csv('region-data-files/gw_region.csv')
tt.info()
tt.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142353 entries, 0 to 142352
Columns: 105 entries, region to all_loan
dtypes: float64(72), int64(26), object(7)
memory usage: 114.0+ MB


Index(['region', 'region_sub', 'apt_name', 'apt_size', 'apt_floor',
       'contract_date', 'contract_price', 'completion_year',
       'period_completed', 'pyung',
       ...
       'so_loan', 'sj_loan', 'us_loan', 'ic_loan', 'jrn_loan', 'jrb_loan',
       'jj_loan', 'ccn_loan', 'ccb_loan', 'all_loan'],
      dtype='object', length=105)